# Create Tracks

Create tracks from passed inner and outer borders. The goal is to automatically generate code that represents the track, i.e., the set of bad states, waypoints as well as the bounding box.


In [79]:
in_inner = 'austria.csv' # filename for the file containing the vertices
in_outer = 'austria.csv' # filename for the file containing the vertices
inner_track_name = 'inner_track' # name for the inner track
inner_track_filename = inner_track_name + '.poly' # filename which contains input for triangle of the inner part of the track
outer_track_name = 'outer_track' # name for the inner track
outer_track_filename = outer_track_name + '.poly' # filename which contains input for triangle of the inner part of the track
triangle_folder = '/home/stefan/tu/repositories/simplex-architectures/cmake-build-release/triangle-prefix/src/triangle/' # triangle-executable folder
hypro_outfile = '/home/stefan/tu/repositories/simplex-architectures/racetracks/austria/bad_states.h' # the file will contain definitions of polytopes for the inner part of the track
hypro_playground = '/home/stefan/tu/repositories/simplex-architectures/racetracks/austria/playground.h' # contains the bounding box

Read in the csv-file containing the vertices of the track.

In [80]:
inner_nodes = []
outer_nodes = []

with open(in_inner, 'r') as f:
    inner_nodes = [ (line.split(",")[0].strip(), line.split(",")[1].strip()) for line in f.readlines() if '#' not in line]

with open(in_outer, 'r') as f:
    outer_nodes = [ (line.split(",")[0].strip(), line.split(",")[1].strip()) for line in f.readlines() if '#' not in line]


Also compute the corners of the bounding box

In [81]:
minx = miny = float('inf')
maxx = maxy = float('-inf')
for v in outer_nodes:
    minx = min(minx, float(v[0]))
    maxx = max(maxx, float(v[0]))
    miny = min(miny, float(v[1]))
    maxy = max(maxy, float(v[1]))

# add a bit of margin for stability
minx -= 1
maxx += 1
miny -= 1
maxy += 1

In [82]:
from platform import node


def writePolyFile(filename,nodes,holes,bounding_box):
    out = open(filename,'w')
    # write first line: number of nodes, dimension, number of attributes, markers (1 or 0)
    out.write(str(len(nodes)+len(bounding_box)) + " 2 0 0\n")
    # write all nodes
    for i in range(len(nodes)):
        out.write(str(i) + " " + str(nodes[i][0]) + " " + str(nodes[i][1]) + "\n")

    for i in range(len(bounding_box)):
        out.write(str(i+len(nodes)) + " " + str(bounding_box[i][0]) + " " + str(bounding_box[i][1]) + "\n")

    # now add segments, first a line indicating how many segments there will be, whether we want to have boundary markers, 
    # then the segments (the two node indices defining a segment)
    out.write(str(len(nodes)+len(bounding_box)) + " 0\n")
    for i in range(len(nodes)):
        out.write(str(i) + " " + str(i) + " " + str((i+1) % len(nodes)) + "\n")
    
    # write segments for bounding box manually, if we have a bounding box
    if len(bounding_box) > 0:
        out.write(str(len(nodes)) + " " + str(len(nodes)) + " " + str((len(nodes)+1)) + "\n")
        out.write(str(len(nodes)+1) + " " + str(len(nodes)+1) + " " + str((len(nodes)+2)) + "\n")
        out.write(str(len(nodes)+2) + " " + str(len(nodes)+2) + " " + str((len(nodes)+3)) + "\n")
        out.write(str(len(nodes)+3) + " " + str(len(nodes)+3) + " " + str((len(nodes))) + "\n")

    # finally, add one line for indicating the number of holes (with no boundary markers)
    out.write(str(len(holes)) + " 0\n")
    # if there are holes, give inner points
    for i in range(len(holes)):
        out.write(str(i) + " " + str(holes[i][0]) + " " + str(holes[i][1]) + "\n")

    out.close()

Prepare a *.poly file for triangle which contains all the nodes and segments, one for the inner and one for the outer track. Note that for the outer track we need to define a hole (the inner track). To do so, we first compute the inner track triangulation, then take a point which is definitely inside one of the computed triangles (the mid-point of the first triangle) and use this point to define the inside of the hole for the outer triangulation. Note that we also need to provide vertices of the bounding box for the outer set.

In [83]:
import os

# write file for inner track part
writePolyFile(inner_track_filename,inner_nodes, [], [])
# call to triangle
os.system(triangle_folder + "triangle -pYj " + inner_track_filename)
# collect triangles
inner_vertex_sets = []
with open(inner_track_name + ".1.ele") as f:
    inner_vertex_sets = [ (int(line.split()[1]), int(line.split()[2]), int(line.split()[3])) for line in f.readlines() if len(line.split()) == 4]

print(str(inner_vertex_sets))

# For the outer track we need to define a hole. A hole is given as a point (inside the hole) surrounded by segments
# We already have the segments, to get an internal point, we try to get an internal point of one of the polytopes
# of the inner set and use this as the basis for our hole (there is only one). To get an inner point, take the average of the first triangle
holes = [ ((float(inner_nodes[inner_vertex_sets[0][0]][0]) + float(inner_nodes[inner_vertex_sets[0][1]][0]) + float(inner_nodes[inner_vertex_sets[0][2]][0]))/3, (float(inner_nodes[inner_vertex_sets[0][0]][1]) + float(inner_nodes[inner_vertex_sets[0][1]][1]) + float(inner_nodes[inner_vertex_sets[0][2]][1]))/3) ]
# collect vertices of the bounding box
bbox = [ (minx,miny),(maxx,miny),(maxx,maxy),(minx,maxy) ]
# write file with holes
writePolyFile(outer_track_filename,outer_nodes, holes, bbox)
# call triangle
os.system(triangle_folder + "triangle -pYj " + outer_track_filename)
# collect triangles
outer_vertex_sets = []
with open(outer_track_name + ".1.ele") as f:
    outer_vertex_sets = [ (int(line.split()[1]), int(line.split()[2]), int(line.split()[3])) for line in f.readlines() if len(line.split()) == 4]

print(str(outer_vertex_sets))


Opening inner_track.poly.
Constructing Delaunay triangulation by divide-and-conquer method.
Delaunay milliseconds:  0
Recovering segments in Delaunay triangulation.
Segment milliseconds:  0
Removing unwanted triangles.
Hole milliseconds:  0

Writing inner_track.1.node.
Writing inner_track.1.ele.
Writing inner_track.1.poly.

Output milliseconds:  0
Total running milliseconds:  0

Statistics:

  Input vertices: 30
  Input segments: 30
  Input holes: 0

  Mesh vertices: 30
  Mesh triangles: 28
  Mesh edges: 57
  Mesh exterior boundary edges: 30
  Mesh interior boundary edges: 0
  Mesh subsegments (constrained edges): 30

[(20, 7, 6), (7, 20, 19), (20, 6, 21), (6, 22, 21), (5, 22, 6), (4, 22, 5), (8, 7, 10), (10, 9, 8), (7, 11, 10), (18, 17, 12), (18, 7, 19), (12, 11, 18), (17, 13, 12), (18, 11, 7), (4, 3, 2), (22, 2, 1), (2, 22, 4), (23, 22, 1), (0, 29, 28), (28, 27, 0), (1, 0, 25), (27, 25, 0), (23, 1, 24), (14, 13, 16), (17, 16, 13), (24, 1, 25), (15, 14, 16), (27, 26, 25)]
Opening oute

Create HyPro-code that creates polytopes for each of the passed triangles.

In [84]:
# add vertices of the bounding box also to the outer nodex, since they will be used later for the file writing
outer_nodes.append((minx,miny))
outer_nodes.append((maxx,miny))
outer_nodes.append((maxx,maxy))
outer_nodes.append((minx,maxy))

In [85]:
def createPolytopeCode(nodes,vertices):
	print("Have vertex indices " + str(vertices))
	return '{\n\
	\tstd::vector<hypro::Point<Number>> points{\n\
	\thypro::Point<Number>({' + str(nodes[vertices[0]][0]) + ',' + str(nodes[vertices[0]][1]) +'}),\n\
	\thypro::Point<Number>({' + str(nodes[vertices[1]][0]) + ',' + str(nodes[vertices[1]][1]) +'}),\n\
	\thypro::Point<Number>({' + str(nodes[vertices[2]][0]) + ',' + str(nodes[vertices[2]][1]) +'}),\n\
	};\n\
	hypro::HPolytope<Number> poly{points};\n\
	res.push_back(hypro::Condition<Number>(poly.matrix(), poly.vector()));\n\
	}'

In [86]:
hyprofile = open(hypro_outfile,'w')

hyprofile.write(
'#include <hypro/representations/GeometricObjectBase.h>\n\
#include <vector>\n\n\
#pragma once\n\n\
namespace simplexArchitectures {\n\
template<typename Automaton>\n\
typename Automaton::conditionVector createBadStates() {\n\
\tusing Number = typename Automaton::NumberType;\n\
\tauto res = typename Automaton::conditionVector();\n\n')

for v_set in inner_vertex_sets:
    hyprofile.write(createPolytopeCode(inner_nodes,v_set))

print(str(len(outer_nodes)))
for v_set in outer_vertex_sets:
    hyprofile.write(createPolytopeCode(outer_nodes,v_set))
    
hyprofile.write('\n\treturn res;\n}\n} // namespace\n')

hyprofile.close()

Have vertex indices (20, 7, 6)
Have vertex indices (7, 20, 19)
Have vertex indices (20, 6, 21)
Have vertex indices (6, 22, 21)
Have vertex indices (5, 22, 6)
Have vertex indices (4, 22, 5)
Have vertex indices (8, 7, 10)
Have vertex indices (10, 9, 8)
Have vertex indices (7, 11, 10)
Have vertex indices (18, 17, 12)
Have vertex indices (18, 7, 19)
Have vertex indices (12, 11, 18)
Have vertex indices (17, 13, 12)
Have vertex indices (18, 11, 7)
Have vertex indices (4, 3, 2)
Have vertex indices (22, 2, 1)
Have vertex indices (2, 22, 4)
Have vertex indices (23, 22, 1)
Have vertex indices (0, 29, 28)
Have vertex indices (28, 27, 0)
Have vertex indices (1, 0, 25)
Have vertex indices (27, 25, 0)
Have vertex indices (23, 1, 24)
Have vertex indices (14, 13, 16)
Have vertex indices (17, 16, 13)
Have vertex indices (24, 1, 25)
Have vertex indices (15, 14, 16)
Have vertex indices (27, 26, 25)
34
Have vertex indices (7, 30, 6)
Have vertex indices (6, 30, 5)
Have vertex indices (18, 19, 20)
Have vert

Create HyPro-code that determines the track playground (i.e., the bounding box)

In [87]:
hyprofile = open(hypro_playground,'w')

hyprofile.write(
'#include <hypro/representations/GeometricObjectBase.h>\n\
#include <vector>\n\n\
#pragma once\n\n\
namespace simplexArchitectures {\n\
template<typename Number>\n\
typename hypro::Box<Number> createPlayground() {\n\
\tusing I = carl::Interval<Number>;\n\
\tusing IV = std::vector<I>;\n\
\treturn hypro::Box<Number>{IV{I{'+str(minx)+","+str(miny)+'},I{'+str(maxx)+','+str(maxy)+'}}};\n}'
)
hyprofile.write('\n} // namespace\n')

hyprofile.close()